### I will be scraping the page from Wikipedia - https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M using BeautifulSoup and it request packages. 

## The first line of action is to take the first 3 items by filtering the the table in the link using class and pahaps 'td'


In [1]:
import requests
from bs4 import BeautifulSoup
r= requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(r.content, "html.parser")
soup = soup.find_all("table", {"class":"wikitable sortable"})[0].find_all("td")
soup[0:3]

[<td>M1A</td>, <td>Not assigned</td>, <td>Not assigned
 </td>]

In [2]:
# We can see from the above output that 1,2 and 3rd tds are for Postcode, Borough and Neighborhood respectively. 
# So that analysis can be effective, transforming the data into pandas dataframe is imminent. 
# We need then to create empty dataframe and data.

In [3]:
import pandas as pd

df = pd.DataFrame(columns= ["Postcode", "Borough","Neighborhood"]) # empty dataframe

for i in range(len(soup)):  # len(soup) stands for number of postcode, borough and neighborhood combinations
     if i % 3==0:  
        df= df.append({'Postcode': soup[i].text, 
                        'Borough': soup[i+1].text, 
                        'Neighborhood': str(soup[i+2].text).replace("\n","")},ignore_index=True)

print("rows, columns:", df.shape)
df.head()

rows, columns: (289, 3)


,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
# the neighborhood has to be the same as borough so we do the following line of code
df.loc[df['Neighborhood'] == 'Not assigned','Neighborhood' ] =df.loc[df['Neighborhood'] == 'Not assigned','Borough' ]
df.iloc[8:9]

,Postcode,Borough,Neighborhood
8,M7A,Queen's Park,Queen's Park


In [5]:
# create and delete index
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
df.reset_index(inplace=True, drop=True)
print("rows, columns:", df.shape)
df.head()

rows, columns: (212, 3)


,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [6]:
# we can a neighborhood with defferent names. so we can put them into one with the following line of codes:
for j in range(1,len(df)):
    if df['Postcode'].iloc[j]==df['Postcode'].iloc[j-1] and df['Borough'].iloc[j]==df['Borough'].iloc[j-1]:
        df['Neighborhood'].iloc[j]=df['Neighborhood'].iloc[j] + ", " + df['Neighborhood'].iloc[j-1]
        

# delete the duplicated row

df['Repeat']=""  # add an artificial column 'Repeat' for duplicated items
for j in range(0,len(df)-1):
    if df['Postcode'].iloc[j]==df['Postcode'].iloc[j+1] and df['Borough'].iloc[j]==df['Borough'].iloc[j+1]:
        df['Repeat'].iloc[j]=1   # type 1 against rows to be deleted

df.drop(df.loc[df['Repeat']==1].index, inplace=True) # delete rows containing 1 under 'Repeat' column
df.drop(['Repeat'], axis=1, inplace=True)   # delete the 'Repeat' column since now there is nothing to do with it
df.reset_index(inplace=True, drop=True)

df.head(11)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [7]:
#To know the number/rows of datafram, we call the shape method
print("rows, columns:", df.shape)

rows, columns: (103, 3)
